# Honours Project - Fake News Detection

In [1]:
"""
git code:
git add .
git commit -m "First commit"
git push origin master
"""

'\ngit code:\ngit add .\ngit commit -m "First commit"\ngit push origin master\n'

This project will be using the approach of CRISP-DM (Cross-industry standard process for data mining), which is a widely used process for knowledge discovery in data sets. 
The process encompasses several phases:

    1. Business Understanding
    2. Data Understanding
    3. Data Preparation
    4. Modeling
    5. Evaluation
    6. Deployment

Possible data sets:

https://www.kaggle.com/pontes/fake-news-sample

Real News: https://archive.ics.uci.edu/ml/datasets/News+Aggregator

https://toolbox.google.com/datasetsearch/search?query=fake%20news&docid=sHyIQgRMuTsFH02AAAAAAA%3D%3D

https://github.com/hanselowski/athene_system/tree/master/data

## Step 1 - Business Understanding

TODO

Goals/ Objectives &
Success Criteria

## Step 2 - Data Understanding

### 2 a) - Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
import sklearn
import string

#nltk.download('punkt')

#from sklearn.model_selection import train_test_split
#import sklearn.model_selection as ms
#import sklearn.feature_extraction.text as text
#import sklearn.naive_bayes as nb
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.naive_bayes import GaussianNB
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score

### 2 b) - Loading the Data Set

In [3]:
df = pd.read_csv("data/FakeNews-(balanced)/fake_or_real_news.csv", encoding="utf-8")
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
#preview of an FAKE article
df.iloc[16,2]

'Shocking! Michele Obama & Hillary Caught Glamorizing Date Rape Promoters First lady claims moral high ground while befriending rape-glorifying rappers Infowars.com - October 27, 2016 Comments \nAlex Jones breaks down the complete hypocrisy of Michele Obama and Hillary Clinton attacking Trump for comments he made over a decade ago while The White House is hosting and promoting rappers who boast about date raping women and selling drugs in their music. \nRappers who have been welcomed to the White House by the Obama’s include “Rick Ross,” who promotes drugging and raping woman in his song “U.O.N.E.O.” \nWhile attacking Trump as a sexual predator, Michelle and Hillary have further mainstreamed the degradation of women through their support of so-called musicians who attempt to normalize rape. NEWSLETTER SIGN UP Get the latest breaking news & specials from Alex Jones and the Infowars Crew. Related Articles'

In [5]:
#preview of an REAL article
df.iloc[8,2]

'Hillary Clinton and Donald Trump made some inaccurate claims during an NBC “commander-in-chief” forum on military and veterans issues:\n\n• Clinton wrongly claimed Trump supported the war in Iraq after it started, while Trump was wrong, once again, in saying he was against the war before it started.\n\n•\xa0Trump said that President Obama set a “certain date” for withdrawing troops from Iraq, when that date was set before Obama was sworn in.\n\n•\xa0Trump said that Obama’s visits to China, Saudi Arabia and Cuba were “the first time in the history, the storied history of Air Force One” when “high officials” of a host country did not appear to greet the president. Not true.\n\n•\xa0Clinton said that Trump supports privatizing the Veterans Health Administration. That’s false. Trump said he supports allowing veterans to seek care at either public or private hospitals.\n\n•\xa0Trump said Clinton made “a terrible mistake on Libya” when she was secretary of State. But, at the time, Trump als

In [6]:
df.info

<bound method DataFrame.info of       Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
5           6903                                        Tehran, USA   
6           7341  Girl Horrified At What She Watches Boyfriend D...   
7             95                  ‘Britain’s Schindler’ Dies at 106   
8           4869  Fact check: Trump and Clinton at the 'commande...   
9           2909  Iran reportedly makes new push for uranium con...   
10          1357  With all three Clintons in Iowa, a glimpse at ...   
11           988  Donald Trump’s Shockingly Weak Delegate Game S...   
12          7041  Strong Solar Storm, Tech Ri

In [7]:
columns = df.columns.tolist()

In [8]:
print(columns)

['Unnamed: 0', 'title', 'text', 'label']


In [9]:
df["label"].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

The data set seems to be well balanced.

In [10]:
type(df["title"])

pandas.core.series.Series

In [11]:
type(df["text"])

pandas.core.series.Series

In [12]:
df.dtypes

Unnamed: 0     int64
title         object
text          object
label         object
dtype: object

It appears that "Unnamed: 0" is the index since it only contains numbers. The column will be checked for duplicates to check.

In [13]:
print(any(df["Unnamed: 0"].duplicated()))

False


In [14]:
df.isnull().values.any()

False

## Step 3 - Data Preparation

### 3 a) - General Polishing

In [15]:
#rename "Unnamed: 0" and make it the index of the data frame
df.columns = ["index", "title", "text", "label"]
df.set_index("index", inplace=True)

In [16]:
df.head()

,title,text,label
index,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [17]:
#order by index
df.sort_index(inplace=True)

In [18]:
df.head()

,title,text,label
index,,,
2,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
3,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
5,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL
6,"Despite Constant Debate, Americans' Abortion O...",It's been a big week for abortion news.\n\nCar...,REAL
7,Obama Argues Against Goverment Shutdown Over P...,President Barack Obama said Saturday night tha...,REAL


In [19]:
df.index

Int64Index([    2,     3,     5,     6,     7,     9,    10,    12,    14,
               16,
            ...
            10543, 10545, 10546, 10547, 10548, 10549, 10551, 10553, 10555,
            10557],
           dtype='int64', name='index', length=6335)

The index seems to skip some numbers, for example 8 and 11. The index will be properly assigned.

In [20]:
df['index'] = df.reset_index().index

In [21]:
df.set_index("index", inplace=True)
df.head()

,title,text,label
index,,,
0,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
1,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
2,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL
3,"Despite Constant Debate, Americans' Abortion O...",It's been a big week for abortion news.\n\nCar...,REAL
4,Obama Argues Against Goverment Shutdown Over P...,President Barack Obama said Saturday night tha...,REAL


It appears that the data contains several characters like \n or \a. They will be removed from the data set.

### 3 b) - Normalising The Data

In [22]:
# the function strip() will be used to remove those characters
# Example:
s = "\n \a abc \n \n"
print(s.strip())

 abc


In [23]:
s = "\n\nCar"
print(s.strip())

Car


In [24]:
df["text"] = df["text"].apply(lambda x: x.strip())

In [25]:
# since some characters are part of the string, they have to be removed with the replace function
df["text"] = df["text"].apply(lambda x: x.replace("\n", ""))
df["text"] = df["text"].apply(lambda x: x.replace("\t", ""))
#df["text"] = df["text"].apply(lambda x: x.replace("\x", ""))
df["text"] = df["text"].apply(lambda x: x.replace("\xa0", ""))

df["title"] = df["title"].apply(lambda x: x.replace("\n", ""))
df["title"] = df["title"].apply(lambda x: x.replace("\t", ""))
#df["title"] = df["title"].apply(lambda x: x.replace("\x", ""))
df["title"] = df["title"].apply(lambda x: x.replace("\xa0", ""))

In [26]:
df.head()

,title,text,label
index,,,
0,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
1,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
2,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL
3,"Despite Constant Debate, Americans' Abortion O...",It's been a big week for abortion news.Carly F...,REAL
4,Obama Argues Against Goverment Shutdown Over P...,President Barack ObamasaidSaturday night that ...,REAL


The next step is to remove punctuation.

In [27]:
#df["text"] = df["text"].apply(lambda x: x.replace(string.punctuation, ""))
#df["title"] = df["title"].apply(lambda x: x.replace(string.punctuation, ""))

df["title"] = df["title"].str.replace("[{}]".format(string.punctuation), "")
df["text"] = df["text"].str.replace("[{}]".format(string.punctuation), "")

In [28]:
#convert every word to lower case - normalising case
df["title"] = df["title"].str.lower()
df["text"] = df["text"].str.lower()
df["label"] = df["label"].str.lower()

In [29]:
df.head()

,title,text,label
index,,,
0,study women had to drive 4 times farther after...,ever since texas laws closed about half of the...,real
1,trump clinton clash in dueling dc speeches,donald trump and hillary clinton now at the st...,real
2,as reproductive rights hang in the balance deb...,washington fortythree years after the supreme...,real
3,despite constant debate americans abortion opi...,its been a big week for abortion newscarly fio...,real
4,obama argues against goverment shutdown over p...,president barack obamasaidsaturday night that ...,real


In the next step stopwords such as "the" or "a" will be removed since they do not contribute to a deeper meaning of a sentence.

In [30]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [31]:
from nltk.tokenize import word_tokenize
import string

#function that tokenises words and removes stop words, punctuation and non alphanumerical characters in a sentence
def func_normalise(sentence):
    tokens = word_tokenize(sentence)
    #print(tokens)
    stop_words = set(stopwords.words("english"))
    stop_words.add("n't")
    stop_words.add("nt")
    
    table = str.maketrans("", "", string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    words = [word for word in stripped if word.isalpha()]
    
    new_sentence = [w for w in words if not w in stop_words] 
            
    new_sentence_str = " ".join(new_sentence)
    
    return new_sentence_str

In [32]:
#testing the function
func_normalise("ever ? since hasn't the / texa*s laws closed about half of the where didn't")

'ever since texas laws closed half'

Now the above function will be applied to the data in order to normalise it.

In [33]:
df["title"] = df["title"].apply(func_normalise)

In [34]:
df["text"] = df["text"].apply(func_normalise)

In [35]:
df.head()

,title,text,label
index,,,
0,study women drive times farther texas laws clo...,ever since texas laws closed half states abort...,real
1,trump clinton clash dueling dc speeches,donald trump hillary clinton starting line gen...,real
2,reproductive rights hang balance debate modera...,washington fortythree years supreme court esta...,real
3,despite constant debate americans abortion opi...,big week abortion newscarly fiorinas passionat...,real
4,obama argues goverment shutdown planned parent...,president barack obamasaidsaturday night congr...,real


As seen in the above example the text data has been (successfully) normalised.

### 3 c) - Stemming 

Stemming is the process of reducing words to their root. For example, "playing" and "played" reduce to the stem "play". Therefore stemming helps with reducing the vocabulary and allows to focus on the sense of a sentence.

In [36]:
#from nltk.stem.porter import PorterStemmer
#according to the nltk website the snowballstemmer is better than the "original" porter stemmer
#https://www.nltk.org/howto/stem.html

from nltk.stem.snowball import SnowballStemmer

#function that stems words in a sentence
def func_stem(sentence):
    tokens = word_tokenize(sentence)
    snowball_stemmer = SnowballStemmer("english")
    stemmed_sentence = [snowball_stemmer.stem(word) for word in tokens]
    stemmed_sentence_str = " ".join(stemmed_sentence)
    return stemmed_sentence_str

In [37]:
#test
func_stem("playing player play played")

'play player play play'

In [38]:
df["title"] = df["title"].apply(func_stem)

In [39]:
df["text"] = df["text"].apply(func_stem)

In [40]:
df.head()

,title,text,label
index,,,
0,studi women drive time farther texa law close ...,ever sinc texa law close half state abort clin...,real
1,trump clinton clash duel dc speech,donald trump hillari clinton start line genera...,real
2,reproduct right hang balanc debat moder drop ball,washington fortythre year suprem court establi...,real
3,despit constant debat american abort opinion r...,big week abort newscar fiorina passion inaccur...,real
4,obama argu gover shutdown plan parenthood,presid barack obamasaidsaturday night congress...,real


In [42]:
df_a = pd.get_dummies(df)

In [43]:
df_a.head()

title_  title_abbi martin expos hillari clinton chair john podesta  \
index                                                                       
0           0                                                  0            
1           0                                                  0            
2           0                                                  0            
3           0                                                  0            
4           0                                                  0            

       title_abedin weiner testifi clinton  \
index                                        
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   

       title_abnorm normal nuclear terror  \
index                                       
0                                       0   
1                                       0   
2                                       0   
3                                       0   
4                                       0   

       title_abort bill drop amid concern femal gop lawmak  \
index                                                        
0                                                      0     
1                                                      0     
2                                                      0     
3                                                      0     
4                                                      0     

       title_abort gaff show trump leftw caricatur conserv  \
index                                                        
0                                                      0     
1                                                      0     
2                                                      0     
3                                                      0     
4                                                      0     

       title_accident case obamacar  \
index                                 
0                                 0   
1                                 0   
2                                 0   
3                                 0   
4                                 0   

       title_accord colleg prof cano symbol cultur appropri  \
index                                                         
0                                                      0      
1                                                      0      
2                                                      0      
3                                                      0      
4                                                      0      

       title_accord reach sander sue dnc suspend access critic voter list  \
index                                                                       
0                                                      0                    
1                                                      0                    
2                                                      0                    
3                                                      0                    
4                                                      0                    

       title_account  ...  \
index                 ...   
0                  0  ...   
1                  0  ...   
2                  0  ...   
3                  0  ...   
4                  0  ...   

       text_yves smith yves real news network segment continu discuss yet anoth acronym trip much econom play field tilt ordinari worker pleas note transcript publish without apostroph ad best could may miss lynn fri welcom real news network lynn fri geneva part seri peter draho explain stori intellectu properti link trade join us australia peter draho professor australian nation univers school regul global govern hold chair intellectu properti queen mari univers london peter draho coauthor in

In [41]:
https://www.sv-europe.com/crisp-dm-methodology/
https://www.kaggle.com/aidenloe/data-understanding-using-python
https://towardsdatascience.com/exploratory-data-analysis-in-python-c9a77dfa39ce
https://towardsdatascience.com/exploratory-data-analysis-tutorial-in-python-15602b417445
https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/


SyntaxError: invalid syntax (<ipython-input-41-16fa96ab1083>, line 1)